 ## Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Change to correct directory

In [2]:
%cd /content/drive/MyDrive/dl4h_project/DynST/
%ls

/content/drive/MyDrive/dl4h_project/DynST
causal.ipynb       dl4hProjectSetup.ipynb  project_causal.ipynb  run.py
config.yaml        multirun/               project_model.ipynb   src/
coxph_model.ipynb  outputs/                pyproject.toml
data/              poetry.lock             README.md


In [ ]:
!pip list -v

In [ ]:
# !curl -sSL https://install.python-poetry.org | python3 -

In [5]:
# !/root/.local/bin/poetry install

Creating virtualenv mimic-NZsxis0q-py3.9 in /root/.cache/pypoetry/virtualenvs
Installing dependencies from lock file

Package operations: 133 installs, 0 updates, 0 removals



In [7]:
# !which python3

/usr/bin/python3


In [9]:
# ! . /root/.cache/pypoetry/virtualenvs/mimic-NZsxis0q-py3.9/bin/activate

In [10]:
# !which python3

/usr/bin/python3


In [ ]:
# !pip list -v

In [4]:
!pip install hydra-core --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 16.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144573 sha256=3d7dd8a7960fec9d2a742007ea64af760ae03f19ecebafde234d9d324b5b3fe8
  Stored in directory: /root/.cache/pip/wheels/23/cf/80/f3efa822e6ab23277902ee9165fe772eeb1dfb8014f359020a
Successfully built antlr4-python3-runtime


install PyTorch lightning

In [3]:
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.6/718.6 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 23.1 MB/s eta 0:00:00


In [4]:
import pytorch_lightning as pl
import torch
import math

from torch.nn import Linear
from src.metric import MeanAbsoluteError, ConcordanceIndex

In [5]:
class DST(pl.LightningModule):
    def __init__(
        self,
        n_codes,
        n_vitals,
        n_demog,
        d_model,
        n_blocks,
        n_heads,
        dropout,
        pad,
        dynamic,
        causal,
        lr=0.001,
        alpha=0.01,
    ):
        super().__init__()
        self.save_hyperparameters()
        self.embed_codes = Linear(n_codes, d_model)
        d1 = 0 if causal else 1
        self.embed_static = Linear(d_model + n_demog + d1, d_model)
        self.embed_vitals = Linear(n_vitals, d_model)
        self.pos_encode = PositionalEncoding(d_model)
        self.pad = pad
        encoder_layer = torch.nn.TransformerEncoderLayer(
            d_model = d_model,
            nhead=n_heads,
            dropout=dropout,
            batch_first=True,
            dim_feedforward=d_model*4
        )
        norm = torch.nn.LayerNorm(d_model)
        self.transformer = torch.nn.TransformerEncoder(encoder_layer, n_blocks, norm)
        d2 = 1 if causal else 0
        self.to_hazard_c = torch.nn.Sequential(
            Linear(d_model + d2, d_model//2),
            torch.nn.ReLU(),
            Linear(d_model//2, 1),
            torch.nn.Sigmoid(),
        )
        self.train_mae = MeanAbsoluteError(pad=pad)
        self.val_mae = MeanAbsoluteError(pad=pad)
        self.val_ci = ConcordanceIndex(pad=pad)
        self.test_mae = MeanAbsoluteError(pad=pad)
        self.test_ci = ConcordanceIndex(pad=pad)

        # how much to weigh MAE loss
        self.alpha = alpha
        self.dynamic = dynamic
        self.causal = causal


    def forward(self, batch):
        # static features
        x = self.embed_codes(batch["codes"]).unsqueeze(1)
        x = self.embed_static(
            torch.cat([x, batch["static"].unsqueeze(1)], 2)
        )
        s = batch["vitals"].shape[1]
        # time-varying features
        if self.dynamic:
            pad_mask = (batch["vitals"][:, :, 0] == self.pad)
            x = x + self.embed_vitals(batch["vitals"])
            # autoregressive mask
            mask = (1 - torch.tril(torch.ones(s, s))).bool().cuda()

        else:
            mask = None
            x = x.repeat(1, s, 1)
            pad_mask = (batch["vitals"][:, :, 0] == self.pad)
        x = self.pos_encode(x)
        x = self.transformer(x, mask, pad_mask)
        if self.causal:
            t = torch.reshape(batch["treatment"], (-1, 1, 1))
            t = t.repeat(1, s, 1)
            # concatenate treatment as a new feature
            x = torch.cat((x, t), 2).float()
        # complement of hazard
        q_hat = self.to_hazard_c(x).squeeze(2)
        s_hat = q_hat.cumprod(1).clamp(min=1e-8)
        return s_hat

    def training_step(self, batch, batch_idx):
        s_hat =  self(batch)
        loss = self.combined_loss(s_hat, batch["survival"])
        self.log("train_loss", loss)
        self.train_mae(s_hat, batch["survival"])
        self.log("train_mae", self.train_mae, on_step=True, on_epoch=False)
        return loss



    def validation_step(self, batch, batch_idx):
        s_hat =  self(batch)
        loss = self.combined_loss(s_hat, batch["survival"])
        self.val_mae.update(s_hat, batch["survival"])
        self.val_ci.update(s_hat, batch["survival"])
        self.log("val_loss", loss)
        self.log("val_mae", self.val_mae, on_step=True, on_epoch=True)
        self.log("val_ci", self.val_ci, on_step=True, on_epoch=True)
        return loss


    def test_step(self, batch, batch_idx):
        s_hat =  self(batch)
        loss = self.combined_loss(s_hat, batch["survival"])
        self.test_mae.update(s_hat, batch["survival"])
        self.test_ci.update(s_hat, batch["survival"])
        self.log("test_mae", self.test_mae, on_step=True, on_epoch=True)
        self.log("test_ci", self.test_ci, on_step=True, on_epoch=True)
        return loss

    def predict_step(self, batch, batch_idx):
        # returns estimated survival times
        s_hat = self(batch)
        mask = (batch["survival"] != self.pad)
        return (s_hat * mask).sum(1)


    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.hparams.lr)

    def ordinal_survival_loss(self, s_hat, y):
        # modified cross entropy loss
        nlog_survival = -torch.log(s_hat)
        nlog_failure = -torch.log(1 - s_hat)
        loss = 0
        loss += nlog_survival * torch.where(y==self.pad, 0, y)
        loss += nlog_failure * torch.where(y==self.pad, 0, (1-y))
        return loss.sum() / (y != self.pad).sum()
    
    def mae_loss(self, s_hat, y):
        observed = (y == 0).any(1).int()
        t_hat = torch.where(y == self.pad, 0, s_hat).sum(1)
        t = torch.where(y == self.pad, 0, y).sum(1)
        zeros = torch.zeros(t.shape).cuda()
        observed_error = torch.abs(t_hat - t) * observed
        censored_error = torch.maximum(zeros, t - t_hat) * (1 - observed)
        return (observed_error.sum() + censored_error.sum()) / t.numel()

    def combined_loss(self, s_hat, y):
        a = self.alpha
        ordinal_loss = self.ordinal_survival_loss(s_hat, y)
        mae_loss = self.mae_loss(s_hat, y)
        return (1 - a) * ordinal_loss + a * mae_loss


In [6]:
class PositionalEncoding(torch.nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = torch.nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(1), :].unsqueeze(0)
        return self.dropout(x)

In [7]:
!pip install hydra-core --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 17.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144573 sha256=08f1f35e0e32e90c4c268100475a1899e5581006b09ea0cdd763c00dc8afb5aa
  Stored in directory: /root/.cache/pip/wheels/23/cf/80/f3efa822e6ab23277902ee9165fe772eeb1dfb8014f359020a
Successfully built antlr4-python3-runtime


In [10]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.1 MB/s eta 0:00:00


In [11]:
# Log in to your W&B account
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [9]:
!python run.py -m model.d_model=32,64 model.alpha=0,0.1,0.2

/content/drive/MyDrive/dl4h_project/DynST/run.py:12: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=".", config_name="config.yaml")
[2023-04-16 19:19:32,399][HYDRA] Launching 6 jobs locally
[2023-04-16 19:19:32,399][HYDRA] 	#0 : model.d_model=32 model.alpha=0
/usr/local/lib/python3.9/dist-packages/hydra/_internal/core_plugins/basic_launcher.py:74: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  ret = run_job(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_preci

# New Section